In [391]:
# parse through contents of assets/courses
from pathlib import Path
path = Path("./assets/courses")

In [1]:
from pycozo import Client

client = Client("sqlite",path="./assets/database_test.db")

In [11]:
client.run("""
?[path, slide_id, sentence_id, sentence] := *slide_sentences[path, slide_id, sentence_id, sentence], *courses_files[path,a,b,course], course="Decision Analysis"
""").head(10)

,path,slide_id,sentence_id,sentence
0,courses/Decision Analysis/Classical Method for...,0,0,"During the last two lectures of our course, w..."
1,courses/Decision Analysis/Classical Method for...,0,1,I will introduce you to another signiﬁcant sub...
2,courses/Decision Analysis/Classical Method for...,0,2,It is concerned with mathematical optimization...
3,courses/Decision Analysis/Classical Method for...,0,3,Multi-objective optimization has been applied ...
4,courses/Decision Analysis/Classical Method for...,0,4,We will start by discussing example multi-obje...
5,courses/Decision Analysis/Classical Method for...,0,5,"Then, we will discuss why these problems are n..."
6,courses/Decision Analysis/Classical Method for...,0,6,We will also introduce some more general conce...
7,courses/Decision Analysis/Classical Method for...,0,7,"Finally, we will discuss a few classical algor..."
8,courses/Decision Analysis/Classical Method for...,0,8,The most important ones include linear scalari...
9,courses/Decision Analysis/Classical Method for...,1,0,"In general, optimization is deﬁned as a proce..."


In [12]:
client.run("""
?[path, start, end, sentence] := *video_sentences[path, start, end, sentence], *courses_files[path,a,b,course], course="Decision Analysis"
""").head(10)

,path,start,end,sentence
0,courses/Decision Analysis/Classical Method for...,0.00,6.75,"During the last two lectures of our course , w..."
1,courses/Decision Analysis/Classical Method for...,6.75,15.66,I will introduce you to another significant su...
2,courses/Decision Analysis/Classical Method for...,15.66,24.82,It is concerned with mathematical optimization...
3,courses/Decision Analysis/Classical Method for...,24.82,35.58,Multiple objective optimization has been appl...
4,courses/Decision Analysis/Classical Method for...,35.58,44.52,We will start by discussing example multi - ob...
5,courses/Decision Analysis/Classical Method for...,44.52,48.99,Then we will discuss why these problems are no...
6,courses/Decision Analysis/Classical Method for...,48.99,57.69,We will also introduce some more general conce...
7,courses/Decision Analysis/Classical Method for...,57.69,63.72,"Finally , we will discuss a few classical algo..."
8,courses/Decision Analysis/Classical Method for...,63.72,75.28,The most important ones include linear scal ar...
9,courses/Decision Analysis/Classical Method for...,75.28,86.78,"In general , optimization is defined as a proc..."


In [16]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [30]:
client.run("""
        ?[start, end, path, sentence] := ~video_sentences:full_text_search {start, end ,path, sentence |
            query: $query,
            k: 1000
        }
        """, {"query": f"ideal point"})

,start,end,path,sentence
0,1092.03,1096.68,courses/Decision Analysis/Classical Method for Multiple Objective Optimization/Decision Analysis Introduction to Multiple Objective Optimization - Classical Optimization Methods.mp4,The par at oph ront is bounded by ideal and n ad ir points .
1,1164.27,1169.56,courses/Decision Analysis/Classical Method for Multiple Objective Optimization/Decision Analysis Introduction to Multiple Objective Optimization - Classical Optimization Methods.mp4,Also for numerical reasons the ideal point is often not used directly .
2,2419.38,2425.33,courses/Decision Analysis/Classical Method for Multiple Objective Optimization/Decision Analysis Introduction to Multiple Objective Optimization - Classical Optimization Methods.mp4,When no DM is available it can be by default set to the ideal point .
3,2439.04,2449.80,courses/Decision Analysis/Classical Method for Multiple Objective Optimization/Decision Analysis Introduction to Multiple Objective Optimization - Classical Optimization Methods.mp4,Moreover when using the ideal point as the reference all P are to optimal solutions can be found no matter if they are supported or not .
4,2666.00,2671.28,courses/Decision Analysis/Introduction to Evolutionary Multiple Objective Optimization/Decision Analysis Introduction to Evolutionary Multiple Objective Optimization.mp4,"On the one hand , it can be run with the reference point set to the ideal one ."
5,2817.51,2823.09,courses/Decision Analysis/Classical Method for Multiple Objective Optimization/Decision Analysis Introduction to Multiple Objective Optimization - Classical Optimization Methods.mp4,First the method is initialized by computing the ideal and native points .


In [393]:
client.run("""
    ?[] <- [[1,3]]
""")

,_0,_1
0,1,3


In [394]:
from enum import Enum

class AssetType(str, Enum):
    VIDEO = "video"
    TEXT = "text"
    PDF = "pdf"
    KADZINSKI_NOTES_PDF = "kadzinski_notes_pdf"

In [395]:
AssetType.VIDEO

<AssetType.VIDEO: 'video'>

In [ ]:
files_relation = {
    "courses_files" : {
        "headers": ["path", "type", "lecture", "course"],
        "rows": []
    }
}

for course in path.iterdir():
    if not course.is_dir():
        continue
    for lecture in course.iterdir():
        if not lecture.is_dir():
            continue
        for asset in lecture.iterdir():
            if not asset.is_file():
                continue
            asset_type = None
            if asset.suffix == ".mp4":
                asset_type = AssetType.VIDEO
            elif asset.suffix == ".txt":
                asset_type = AssetType.TEXT
            elif asset.suffix == ".pdf":
                if asset.stem.endswith("-notes"):
                    asset_type = AssetType.KADZINSKI_NOTES_PDF
                else:
                    asset_type = AssetType.PDF
            if asset_type is not None:
                files_relation["courses_files"]["rows"].append([str(asset.relative_to("assets").as_posix()), asset_type.value, lecture.name, course.name])
            else:
                print("Unknown asset type: ", asset)

files_relation

In [398]:
client.run("::remove courses_files")
def create_course_files_relation(client):
    return client.run("""
    :create courses_files {path => type, lecture, course}
    """)
create_course_files_relation(client)

,status
0,OK


In [399]:
client.import_relations(files_relation)

In [400]:
client.run("::remove reference_assets")
def create_reference_assets_relation(client):
    return client.run("""
    :create reference_assets {reference, referred_to, type}
""")
create_reference_assets_relation(client)

,status
0,OK


In [401]:
kadzinski_notes = client.run("""
    ?[path, type, lecture, course] := *courses_files[path, type, lecture, course], type == "kadzinski_notes_pdf"
""")
kadzinski_notes

,path,type,lecture,course
0,courses/Decision Analysis/Classical Method for...,kadzinski_notes_pdf,Classical Method for Multiple Objective Optimi...,Decision Analysis
1,courses/Decision Analysis/Game Theory - Conges...,kadzinski_notes_pdf,Game Theory - Congestion and Extensive Games,Decision Analysis
2,courses/Decision Analysis/Game Theory - Soluti...,kadzinski_notes_pdf,Game Theory - Solution Concepts in Strategic G...,Decision Analysis
3,courses/Decision Analysis/Introduction to Evol...,kadzinski_notes_pdf,Introduction to Evolutionary Multiple Objectiv...,Decision Analysis
4,courses/Decision Analysis/Introduction to Soci...,kadzinski_notes_pdf,Introduction to Social Choice Theory - Group D...,Decision Analysis
5,courses/Decision Analysis/Voting Rules - Chara...,kadzinski_notes_pdf,"Voting Rules - Characterization Results, Strat...",Decision Analysis


In [402]:
def assign_slides_to_kadzinski_notes(client, slides_path:Path, notes_path:Path):
    return client.run("""
            ?[reference, referred_to, type] <- [[$reference, $referred_to, $type]]
            :put reference_assets {reference, referred_to, type}
        """, {"reference": slides_path.as_posix(), "referred_to": notes_path.as_posix(), "type": "slides"})

In [403]:
import fitz
import re



In [404]:
import json
def write_array_to_file(path, array):
    with open(path, "w") as f:
        json.dump(array, f)


In [405]:
def read_pdf_kadzinski_notes(kadzinski_notes_path):
    kadzinski_notes_pattern = re.compile(r"\[[0-9]+\]")
    doc = fitz.open("./assets/"+kadzinski_notes_path)
    notes = []
    for page in doc:
        page_text = page.get_text()
        for match in kadzinski_notes_pattern.finditer(page_text):
                notes.append(page_text[match.end() :])
    return notes
notes = read_pdf_kadzinski_notes(kadzinski_notes.path[0])

In [406]:
def assign_text_to_pdf(client, slides_path:Path, text_path:Path):
    return client.run("""
            ?[reference, referred_to, type] <- [[$reference, $referred_to, $type]]
            :put reference_assets {reference, referred_to, type}
        """, {"reference": text_path.as_posix(), "referred_to": slides_path.as_posix(), "type": "text"})

In [407]:
assets_path = Path("./assets")
for row in kadzinski_notes.itertuples():

    
    slides_path = Path(row.path.replace("-notes",""))

    if (assets_path / slides_path).exists():
        try:
            assign_slides_to_kadzinski_notes(client, slides_path, Path(row.path))
        except pycozo.client.QueryException as e:
            print("Error while assigning slides for slide_notes:", e)
    
    notes_path = Path("processed") / Path(row.path).with_suffix(".json").name

    if not (assets_path / notes_path).exists():
        notes = read_pdf_kadzinski_notes(Path(row.path))
        # write notes as csv
        write_array_to_file(notes_path, notes)

    try:
        assign_text_to_pdf(client, Path(row.path), notes_path)
    except pycozo.client.QueryException as e:
        print("Error while assigning text for slide_notes:", e)


In [408]:
client.run("""
    ?[reference, referred_to, type] := *reference_assets[reference, referred_to, type] """)

,reference,referred_to,type
0,courses/Decision Analysis/Classical Method for...,courses/Decision Analysis/Classical Method for...,slides
1,courses/Decision Analysis/Game Theory - Conges...,courses/Decision Analysis/Game Theory - Conges...,slides
2,courses/Decision Analysis/Game Theory - Soluti...,courses/Decision Analysis/Game Theory - Soluti...,slides
3,courses/Decision Analysis/Introduction to Evol...,courses/Decision Analysis/Introduction to Evol...,slides
4,courses/Decision Analysis/Introduction to Soci...,courses/Decision Analysis/Introduction to Soci...,slides
5,courses/Decision Analysis/Voting Rules - Chara...,courses/Decision Analysis/Voting Rules - Chara...,slides
6,processed/da-lec10-notes.json,courses/Decision Analysis/Voting Rules - Chara...,text
7,processed/da-lec11-notes.json,courses/Decision Analysis/Classical Method for...,text
8,processed/da-lec12-notes.json,courses/Decision Analysis/Introduction to Evol...,text
9,processed/da-lec7-notes.json,courses/Decision Analysis/Game Theory - Soluti...,text


In [409]:
pdfs = client.run("""
    ?[path, type, lecture, course] := *courses_files[path, type, lecture, course], type == "pdf"
""")
pdfs

,path,type,lecture,course
0,courses/Big Data and Distributed Processing/Bi...,pdf,Big Data and noSQL models,Big Data and Distributed Processing
1,courses/Big Data and Distributed Processing/CA...,pdf,CAP theorem and BigTable,Big Data and Distributed Processing
2,courses/Big Data and Distributed Processing/Ca...,pdf,Cassandra,Big Data and Distributed Processing
3,courses/Big Data and Distributed Processing/Co...,pdf,Consensus and Paxos algorithm,Big Data and Distributed Processing
4,courses/Big Data and Distributed Processing/Dy...,pdf,Dynamo,Big Data and Distributed Processing
5,courses/Big Data and Distributed Processing/In...,pdf,Introduction,Big Data and Distributed Processing
6,courses/Big Data and Distributed Processing/Lo...,pdf,Logical time and broadcast protocols,Big Data and Distributed Processing
7,courses/Big Data and Distributed Processing/Mo...,pdf,Models of distributed systems,Big Data and Distributed Processing
8,courses/Big Data and Distributed Processing/Or...,pdf,Ordering of distributed events and logical time,Big Data and Distributed Processing
9,courses/Big Data and Distributed Processing/Sp...,pdf,Spark,Big Data and Distributed Processing


In [410]:
def read_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = []
    for page in doc:
        text.append(page.get_text("text"))
    return text

In [411]:
for row in pdfs.itertuples():

    pdf_text_filename =  Path(row.path).with_suffix(".json").name
    pdf_text_path = Path("./assets/processed") / pdf_text_filename
    if (Path("./assets") / pdf_text_path).exists():
        pdf_path = Path("./assets") / Path(row.path)
        print("Processing pdf:", row.path)
        pdf_text = read_pdf_text(pdf_path)
        write_array_to_file(pdf_text_path, pdf_text)
    
    try:
        assign_text_to_pdf(client, Path(row.path), Path("processed") / pdf_text_filename)
    except pycozo.client.QueryException as e:
        print("Error while assigning text for pdf:", e)
    

In [ ]:
# for each file in assets/processed

processed_files = client.run("""
    ?[reference, referred_to] := *reference_assets[reference, referred_to, "text"]
""")
processed_files

In [413]:
import nltk
from nltk import sent_tokenize
from pprint import pprint

In [ ]:
slide_sentences_relation = {
    "slide_sentences" : {
        "headers": ["path", "slide_id", "sentence_id", "sentence"],
        "rows": []
    }
}

for row in processed_files.itertuples():
    path = row.reference
    
    if path.endswith(".json"):
        paragraphs = json.load(open("./assets/"+path, "r"))
        for slide_id,paragraph in enumerate(paragraphs):
            sentences = sent_tokenize(paragraph.replace("\n"," "))
            for sentence_id,sentence in enumerate(sentences):
                slide_sentences_relation["slide_sentences"]["rows"].append([path, slide_id, sentence_id, sentence])

slide_sentences_relation

In [418]:
#client.run("::remove slide_sentences")
def create_slide_sentences_relation(client):
    return client.run("""
    :create slide_sentences {path, slide_id, sentence_id => sentence}
    """)
create_slide_sentences_relation(client)

,status
0,OK


In [419]:
client.import_relations(slide_sentences_relation)

In [420]:
client.run("""
    ?[path, slide_id, sentence_id, sentence] := *slide_sentences[path, slide_id, sentence_id, sentence]
""")

,path,slide_id,sentence_id,sentence
0,processed/2023_AI_BDDS_L1.json,0,0,Big Data & Distributed Processing Lecture 1: ...
1,processed/2023_AI_BDDS_L1.json,0,1,inż.
2,processed/2023_AI_BDDS_L1.json,0,2,"Anna Kobusińska, prof. PP Anna.Kobusinska@cs.p..."
3,processed/2023_AI_BDDS_L1.json,1,0,Course structure ▪ Course staff: ▪ Dr hab.
4,processed/2023_AI_BDDS_L1.json,1,1,inż.
...,...,...,...,...
4487,processed/da-lec9.json,43,1,Dodgson rule: elect alternative X that minimiz...
4488,processed/da-lec9.json,44,0,Classiﬁcation of Voting Rules and Electoral Sy...
4489,processed/da-lec9.json,45,0,Voting Rules - Summary § There exist many dif...
4490,processed/da-lec9.json,45,1,Summary Comparison of voting rules via s What...


In [421]:
client.run("""
    ::fts create slide_sentences:full_text_search {
    extractor: sentence,
    tokenizer: Simple,
    filters: [Lowercase,AlphaNumOnly, Stemmer('English')],
}
""")

,status
0,OK


In [422]:
client.run("""
        ?[slide_id, path, sentence] := ~slide_sentences:full_text_search {slide_id,path, sentence |
            query: $query,
            k: 1000
        }
        """, {"query": f"NEAR/1(transferable vote)"})

,slide_id,path,sentence
0,15,processed/da-lec9-notes.json,"Whichever the regulations, when such multiple ..."
1,15,processed/da-lec9.json,Multiple Non-transferable Vote § Winners usua...
2,16,processed/da-lec9-notes.json,"Then, it is called the single non-transferable..."
3,16,processed/da-lec9.json,Single Non-transferable Vote § Single non-tra...
4,20,processed/da-lec9-notes.json,The one in the spirit of IRV is called Single ...
5,20,processed/da-lec9.json,Single Transferable Vote - Hare-Clark System (...
6,21,processed/da-lec9.json,Single Transferable Vote - Hare-Clark System (...
7,22,processed/da-lec9.json,Single Transferable Vote - Characteristics § ...
8,44,processed/da-lec9-notes.json,"Some of the rules we discussed, including lim..."
9,44,processed/da-lec9.json,Classiﬁcation of Voting Rules and Electoral Sy...


In [ ]:
results = client.run(
        """
        results[slide_id, path, sentence] := ~slide_sentences:full_text_search {slide_id,path, sentence |
            query: $query,
            k: 1000
        }
        ?[slide_id, referred_to, sentence, referred_type] := results[slide_id, path, sentence], *reference_assets[reference, referred_to, asset_type], reference=path, *courses_files[referred_to, referred_type, lecture, course]

        """,
        {"query": "social choices"},
    )

results


In [37]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util

bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256
top_k = 32

In [3]:
sentences = client.run("""
    ?[slide_id,sentence_id, path, sentence] := *slide_sentences[path, slide_id, sentence_id, sentence]
    :order path, slide_id, sentence_id
    """).groupby(["path","slide_id"])

In [25]:
len("Hello. my name is Philipp and I'm from Germany hi fsadfsadafadf df sadfafsdfasffs fsf dsafsdfasfdfdsa sdfasfffa fafd safs")

121

In [24]:
bi_encoder.tokenize("Hello. my name is Philipp and I'm from Germany hi fsadfsadafadf df sadfafsdfasffs fsf dsafsdfasfdfdsa sdfasfffa fafd safs")["input_ids"].shape

torch.Size([121, 3])

In [80]:
# for each slide

#passages dict with passage, path, slide_id, start_sentence_id, end_sentence_id

passages = []
window_size = 3
max_len = 0
count = 0
for name, group in sentences:

    candidate_passage = ""
    end_idx = 0
    for start_idx in range(0, len(group)):
        candidate_passage = ""
        end_idx = start_idx
        next_passage = group.sentence.iloc[end_idx]
        
        while len(next_passage) < 256 and end_idx < len(group):
            candidate_passage = next_passage
            next_passage += group.sentence.iloc[end_idx]
            end_idx = min(end_idx + 1, len(group))
        if len(candidate_passage) == 0:
            end_idx = start_idx
            candidate_passage = group.sentence.iloc[start_idx]

        #print(candidate_passage)
        passages.append({
            "passage": candidate_passage,
            "path": group.path.iloc[0],
            "slide_id": group.slide_id.iloc[0],
            "start_sentence_id": start_idx,
            "end_sentence_id": end_idx
        })
        
        if end_idx == len(group):
            break


print(count,max_len)
print("Number of passages:", len(passages))

    


0 0
Number of passages: 3695


In [83]:
from tqdm import tqdm
for passage in tqdm(passages, desc="Embedding passages", total=len(passages)):
    passage["embedding"] = bi_encoder.encode(passage["passage"])

Embedding passages: 100%|██████████| 3695/3695 [01:37<00:00, 37.98it/s]


In [85]:
passages[0]["embedding"].shape

(384,)

In [91]:
# client.run("::remove passages")
client.run("""
    :create passages {path:String, slide_id:Int, start_sentence_id:Int, end_sentence_id:Int => embedding:<F32;384>}
""")

,status
0,OK


In [ ]:
passage_relation = {
    "passages" : {
        "headers": ["path", "slide_id", "start_sentence_id", "end_sentence_id", "embedding"],
        "rows": []
    }
}
for passage in passages:
    passage_relation["passages"]["rows"].append([passage["path"], passage["slide_id"], passage["start_sentence_id"], passage["end_sentence_id"], passage["embedding"].tolist()])
passage_relation["passages"]["rows"][9]

In [101]:
client.import_relations(passage_relation)

In [35]:
client.run("""
    ?[path, start, end, embedding] := *passages_videos[path, start, end, embedding]
""")

,path,start,end,embedding
0,courses/Decision Analysis/Classical Method for Multiple Objective Optimization/Decision Analysis Introduction to Multiple Objective Optimization - Classical Optimization Methods.mp4,0.00,24.82,"[0.03525041416287422, -0.012757725082337856, 0.06420788913965225, -0.02900460734963417, 0.007578068412840366, 0.02146882936358452, -0.04705094173550606, -0.0651724562048912, 0.06426721811294556, -0.0543876476585865, -0.06539765000343323, 0.022398239001631737, 0.0078124795109033585, 0.018473224714398384, 0.03979523852467537, -0.016209600493311882, -0.08571882545948029, 0.01713414490222931, -0.04305701330304146, -0.0478997603058815, 0.0035339591559022665, -0.004887785296887159, 0.0069581568241119385, 0.0016585461562499404, -0.06523644924163818, 0.05097775161266327, -0.05357888713479042, -0.08281950652599335, -0.005811094772070646, -0.023694563657045364, -0.07523605227470398, 0.04502827674150467, -0.031683262437582016, -0.0734313428401947, -0.051994405686855316, 0.05315699055790901, -0.10266800969839096, 0.02686411328613758, 0.01930694468319416, 0.06863519549369812, -0.06788777559995651, 0.02670576050877571, -0.03298872336745262, -0.020809831097722054, 0.017149096354842186, -0.03586661070585251, -0.052509136497974396, -0.07991757988929749, 0.03386393561959267, -0.030867764726281166, -0.02784939669072628, -0.024619894102215767, -0.11647555977106094, -0.009394421242177486, 0.008305558003485203, 0.03400140628218651, -0.02387530542910099, 0.038053520023822784, -0.05961454659700394, 0.04341660812497139, -0.02776297554373741, -0.02445101924240589, -0.0939660519361496, 0.020849090069532394, 0.11836672574281693, 0.018954915925860405, -0.029705822467803955, 0.00614077877253294, 0.06683571636676788, 0.06160837411880493, -0.0003842009755317122, -0.018050529062747955, 0.011551585979759693, 0.03312712162733078, 0.04185076430439949, 0.017620058730244637, -0.011594519950449467, 0.040180791169404984, 0.0030031432397663593, -0.009878776036202908, 0.048089537769556046, -0.012626433745026588, -0.010948543436825275, -0.026685360819101334, 0.03615114465355873, -0.057025451213121414, -0.04417770728468895, -0.09537645429372787, 0.03483370319008827, 0.025493374094367027, -0.0329953134059906, -0.10637620091438293, -0.03549933806061745, 0.0817522332072258, -0.11536993831396103, 0.029962243512272835, -0.10227108001708984, 0.03644087165594101, 0.12299028038978577, 0.012489739805459976, ...]"
1,courses/Decision Analysis/Classical Method for Multiple Objective Optimization/Decision Analysis Introduction to Multiple Objective Optimization - Classical Optimization Methods.mp4,6.75,35.58,"[0.055108100175857544, -0.07808085530996323, -0.03425076603889465, -0.09597914665937424, -0.04517660662531853, -0.06845192611217499, 0.020611781626939774, 0.07126545161008835, -0.026486394926905632, 0.05038564279675484, -0.03533082455396652, -0.06849964708089828, 0.03794747218489647, 0.0529651865363121, -0.026249194517731667, 0.0503087155520916, 0.016741270199418068, 0.052772630006074905, -0.04738885536789894, -0.051676880568265915, 0.10971768200397491, -0.0017742651980370283, -0.005128263495862484, -0.058310672640800476, -0.015210222452878952, 0.07675137370824814, 0.009084632620215416, -0.014117700047791004, 0.014741218648850918, -0.03145952895283699, 0.03980432078242302, 0.08588284999132156, 0.08294997364282608, -0.010092724114656448, 0.02941691316664219, 0.06291116029024124, -0.053349997848272324, 0.08369683474302292, 0.0005401836824603379, -0.019027214497327805, -0.031046705320477486, -0.012719591148197651, 0.04734351858496666, 0.0023285802453756332, 0.02374504692852497, -0.046908408403396606, -0.09632673859596252, 0.02110155113041401, 0.028302574530243874, 0.036246057599782944, -0.01923232525587082, -0.06275375932455063, -0.0858406126499176, -0.012308821082115173, 0.0711420401930809, 0.03252492845058441, -0.018742144107818604, -0.029828857630491257, 0.003752851625904441, 0.003616537433117628, -0.008961145766079426, 0.05448559671640396, 0.04115559533238411

In [103]:
client.run("""
    ::hnsw create passages:semantic {
    dim: 384,
    m: 50,
    dtype: F32,
    fields: [embedding],
    distance: Cosine,
    ef_construction: 20,
    extend_candidates: false,
    keep_pruned_connections: false,
}
""")

,status
0,OK


In [ ]:
for passage in passages

In [79]:
query_embedding = bi_encoder.encode("how to find non-dominated solutions?")

In [83]:
results = client.run("""
    ?[dist, path, start, end] := ~passages_videos:semantic{ path, start,end, embedding |
        query: q,
        k: 10,
        ef: 50,
        bind_distance: dist,
    }, q = vec($query)
    :order dist
""", {"query": query_embedding.tolist()})

In [85]:
path, start, end= results.path[1], results.start[1], results.end[1]

client.run("""?[path, start,end,sentence] := *video_sentences[path, start, end, sentence], path=$path, start >= $start, end <= $end """,
            {
                "path": path,
                "start": start,
                "end": end
            },
        ).sentence.to_list()

['To determine the non - dominated solutions in the current population , we can apply the algorithm performed by K unk and others in 1975 .',
 'It is based on a recurrent call of function R .']

[' Since, in practice, the entire Pareto front is unknown, the nadir point can only be approximated.',
 'It is much easier to determine the max point with the worst objective values of the entire objective space.',
 'Therefore, it is often used as an estimate of the nadir point.']

In [ ]:
bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

In [318]:
for result in results.to_dict(orient="records"):
    print(result["slide_id"], result["sentence"])

0 These include game theory, social choice theory, and multiple  objective optimization.
0  In the following two lectures, we will talk about social choice theory.
0 If you remembered the list of the recent hot topics in Artiﬁcial Intelligence, the computational social choice was in there.
0 Decision Analysis Introduction to social choice theory:  group decision making by voting  Miłosz Kadziński  Institute of Computing Science  Poznan University of Technology, Poland  Artiﬁcial Intellience Decision Analysis
0  In the previous lecture devoted to the social choice theory, you learned a few tens of voting rules and electoral systems.
1 Logic:  modeling the reasoning patterns  of agents engaging in strategic  interaction Mathematics:  infinite games (set theory) Economy:  Linguistics Biology Game theory Game theory interaction Philosophy:  analysis of the conflicts arising  between what people ought to do  and what they actually do (ethics) Economy:  modeling competing behaviors  of inter

In [ ]:
#todo videos